In [1]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
import datetime as dt

In [2]:
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:////tmp/test.db'
db = SQLAlchemy(app)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/flask_sqlalchemy/__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(


In [3]:
class User(db.Model):
    id = db.Column(db.Text, primary_key=True, nullable=False)
    username = db.Column(db.String(30), unique=True, nullable=False)
    display_name = db.Column(db.String(30))
    first_name = db.Column(db.String(30), nullable=False)
    last_name = db.Column(db.String(30), nullable=False)

    def __repr__(self):
        return '<User(username=%s)>' % self.username

class Post(db.Model):
    id = db.Column(db.Integer, primary_key=True, nullable=False)

    creator_id = db.Column(db.Text, db.ForeignKey('user.id'), nullable=False)
    user = db.relationship("User", backref=db.backref('posts', lazy=True, cascade='all, delete-orphan'))

    created_at = db.Column(db.DateTime(), default=dt.datetime.now(), nullable=False)
    title = db.Column(db.String(140), nullable=False)
    body = db.Column(db.Text, nullable=False)
    num_votes = db.Column(db.Integer, default=0)
    
    def __repr__(self):
        return '<Post(creator_id=%s, title=%s)>' % (self.creator_id, self.title)

In [4]:
class Comment(db.Model):
    id = db.Column(db.Integer, primary_key=True, nullable=False)

    created_at = db.Column(db.DateTime(), default=dt.datetime.now(), nullable=False)
    body = db.Column(db.Text, nullable=False)

    creator_id = db.Column(db.Text, db.ForeignKey('user.id'), nullable=False)
    user = db.relationship('User', backref=db.backref('comments', lazy=True, cascade='all, delete-orphan'))

    post_id = db.Column(db.Integer, db.ForeignKey('post.id'), nullable=False)
    post = db.relationship('Post', backref=db.backref('comments', lazy=True, order_by=created_at, cascade='all, delete-orphan'))

    # dunno if we'll use this, putting it here in case we do so we don't have to run a migration
    num_votes = db.Column(db.Integer, default=0)

    def __repr__(self):
        return '<Comment(creator_id=%s, post_id=%s>' % (self.creator_id, self.post_id)

In [5]:
db.create_all()

In [8]:
Comment.__table__.columns.keys()

['id', 'created_at', 'body', 'creator_id', 'post_id', 'num_votes']